In [1]:
import re

from oauth2client.service_account import ServiceAccountCredentials
import gspread
import string
from arabic_reshaper import ArabicReshaper

In [2]:
import os

import openai

In [3]:
from dotenv import load_dotenv
# Load your API key from an environment variable or secret management service
load_dotenv()
openai.api_key = os.environ["OPENAPI_KEY_THAWRA"]

In [4]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds_sheet = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json', scope)
client = gspread.authorize(creds_sheet)

# Fetch PDF URLs from Google Sheets document
spreadsheet_key = '1p64cFLW4g6ENO8i6462NkORnYFPThZxBK8mscPgl_Vg'
sheet_name = 'Main'
worksheet = client.open_by_key(spreadsheet_key).worksheet(sheet_name)

In [5]:
texts = worksheet.col_values(160)
# texts = [x for x in texts if x and x !='drive_text1'] 
len(texts)

3710

In [6]:
texts = list(dict.fromkeys(texts))
len(texts)

2119

In [7]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [8]:
""""here is an example how to do it :
    example text : "اﻋﺗﺻﺎمﻟﻣوظﻔﻲﻣﺳﺗﺷﻔﻰﺻﯾدااﻟﺣﻛوﻣﻲاﺣﺗﺟﺎﺟﺎﻋﻠﻰﻋدمدﻓﻊﻣﺳﺗﺣﻘﺎﺗﮭم اﻋﺘﺼﺎم ﻟﻤﻮﻇﻔﻲ ﻣﺴﺘﺸﻔﻰ ﺻﻴﺪا اﻟﺤﻜﻮﻣﻲ اﺣﺘﺠﺎﺟﺎً ﻋﻠﻰ ﻋﺪم دﻓﻊ ﻣﺴﺘﺤﻘﺎﺗﻬﻢ اﻟﻮﻛﺎﻟﺔ اﻟﻮﻃﻨﻴﺔ ﻟﻺﻋﻼم اﻟﻤﺼﺪر أﻣﺎم ﺑﺎﺣﺔ ﻣﺪﺧﻞ اﻟﻄﻮارىء اﺣﺘﺠﺎﺟﺎً ﻋﻠﻰ ﻋﺪم دﻓﻊ رواﺗﺒﻬﻢ ﻣﻨﺬ ﻣﺴﺘﺸﻔﻰ ﺻﻴﺪا اﻟﺤﻜﻮﻣﻲ اﻋﺘﺼﻢ ﻣﻮﻇﻔﻮ ﺧﻤﺴﺔ أﺷﻬﺮ وﺗﻀﺎﻣﻨﺎً ﻣﻊ زﻣﻴﻠﻬﻢ ﻣﺤﻤﺪ ﻣﺼﻄﻔﻰ اﻟﻤﻀﺮب ﻋﻦ اﻟﻄﻌﺎم ﻣﻨﺬ ﻳﻮﻣﻴﻦ رﺋﻴﺲ اﻟﺤﻜﻮﻣﺔ ﺳﻌﺪ اﻟﺤﺮﻳﺮي اﺑﻦ وﻧﺎﺷﺪ رﺋﻴﺲ ﻟﺠﻨﺔ ﻣﺘﺎﺑﻌﺔ اﻟﻤﻮﻇﻔﻴﻦ ﻓﻲ اﻟﻤﺴﺘﺸﻔﻰ ﺧﻠﻴﻞ ﻛﺎﻋﻴﻦ، ﻓﻲ ﺑﻴﺎن، ﺻﻴﺪا ووزﻳﺮ اﻟﺼﺤﺔ ﺟﻤﻴﻞ ﺟﺒﻖ واﻟﻤﺴﺆوﻟﻴﻦ اﻟﻤﻌﻨﻴﻴﻦ وﻓﺎﻋﻠﻴﺎت ﻣﺪﻳﻨﺔ ﺻﻴﺪا ﻟﻠﺘﺪﺧﻞ واﻧﺼﺎف اﻟﻤﻮﻇﻔﻴﻦ ودﻓﻊ ﻟﻢ ﻳﺴﺘﻄﻴﻌﻮا اﻟﺪوام ، ﻣﺸﻴﺮاً إﻟﻰ أنّ اﻟﻤﻮﻇﻔﻴﻦ رواﺗﺒﻬﻢ اﻟﻤﺴﺘﺤﻘﺔ وﻋﺪم اﻟﺴﻤﺎح ﺑﺈﻗﻔﺎل اﻟﻤﺴﺘﺸﻔﻰ "
    cleaned text : " اﻋﺗﺻﺎم ﻟﻣوظﻔﻲ ﻣﺳﺗﺷﻔﻰ ﺻﯾدا اﻟﺣﻛوﻣﻲ اﺣﺗﺟﺎﺟﺎً ﻋﻠﻰ ﻋدم دﻓﻊ ﻣﺳﺗﺣﻘﺎﺗﮭم أﻣﺎم ﺑﺎﺣﺔ ﻣﺪﺧﻞ اﻟﻄﻮارىءاﺣﺘﺠﺎﺟﺎً ﻋﻠﻰ ﻋﺪم دﻓﻊ رواﺗﺒﻬﻢ ﻣﻨﺬ ﺧﻤﺴﺔ أﺷﻬﺮ وﺗﻀﺎﻣﻨﺎً ﻣﻊ زﻣﻴﻠﻬﻢ ﻣﺤﻤﺪ ﻣﺼﻄﻔﻰ اﻟﻤﻀﺮب ﻋﻦ اﻟﻄﻌﺎم ﻣﻨﺬ ﻳﻮﻣﻴﻦ رﺋﻴﺲ اﻟﺤﻜﻮﻣﺔ ﺳﻌﺪ اﻟﺤﺮﻳﺮي اﺑﻦ وﻧﺎﺷﺪ رﺋﻴﺲ ﻟﺠﻨﺔ ﻣﺘﺎﺑﻌﺔ اﻟﻤﻮﻇﻔﻴﻦ ﻓﻲ اﻟﻤﺴﺘﺸﻔﻰ ﺧﻠﻴﻞ ﻛﺎﻋﻴﻦ، ﻓﻲ ﺑﻴﺎن، ﺻﻴﺪا ووزﻳﺮ اﻟﺼﺤﺔ ﺟﻤﻴﻞ ﺟﺒﻖ واﻟﻤﺴﺆوﻟﻴﻦ اﻟﻤﻌﻨﻴﻴﻦ وﻓﺎﻋﻠﻴﺎت ﻣﺪﻳﻨﺔ ﺻﻴﺪا ﻟﻠﺘﺪﺧﻞ واﻧﺼﺎف اﻟﻤﻮﻇﻔﻴﻦ ودﻓﻊ ﻟﻢ ﻳﺴﺘﻄﻴﻌﻮا اﻟﺪوام ، ﻣﺸﻴﺮاً إﻟﻰ أنّ اﻟﻤﻮﻇﻔﻴﻦ رواﺗﺒﻬﻢ اﻟﻤﺴﺘﺤﻘﺔ وﻋﺪم اﻟﺴﻤﺎح ﺑﺈﻗﻔﺎل اﻟﻤﺴﺘﺸﻔﻰ "
    Now """

'"here is an example how to do it :\n    example text : "اﻋﺗﺻﺎمﻟﻣوظﻔﻲﻣﺳﺗﺷﻔﻰﺻﯾدااﻟﺣﻛوﻣﻲاﺣﺗﺟﺎﺟﺎﻋﻠﻰﻋدمدﻓﻊﻣﺳﺗﺣﻘﺎﺗﮭم اﻋﺘﺼﺎم ﻟﻤﻮﻇﻔﻲ ﻣﺴﺘﺸﻔﻰ ﺻﻴﺪا اﻟﺤﻜﻮﻣﻲ اﺣﺘﺠﺎﺟﺎً ﻋﻠﻰ ﻋﺪم دﻓﻊ ﻣﺴﺘﺤﻘﺎﺗﻬﻢ اﻟﻮﻛﺎﻟﺔ اﻟﻮﻃﻨﻴﺔ ﻟﻺﻋﻼم اﻟﻤﺼﺪر أﻣﺎم ﺑﺎﺣﺔ ﻣﺪﺧﻞ اﻟﻄﻮارىء اﺣﺘﺠﺎﺟﺎً ﻋﻠﻰ ﻋﺪم دﻓﻊ رواﺗﺒﻬﻢ ﻣﻨﺬ ﻣﺴﺘﺸﻔﻰ ﺻﻴﺪا اﻟﺤﻜﻮﻣﻲ اﻋﺘﺼﻢ ﻣﻮﻇﻔﻮ ﺧﻤﺴﺔ أﺷﻬﺮ وﺗﻀﺎﻣﻨﺎً ﻣﻊ زﻣﻴﻠﻬﻢ ﻣﺤﻤﺪ ﻣﺼﻄﻔﻰ اﻟﻤﻀﺮب ﻋﻦ اﻟﻄﻌﺎم ﻣﻨﺬ ﻳﻮﻣﻴﻦ رﺋﻴﺲ اﻟﺤﻜﻮﻣﺔ ﺳﻌﺪ اﻟﺤﺮﻳﺮي اﺑﻦ وﻧﺎﺷﺪ رﺋﻴﺲ ﻟﺠﻨﺔ ﻣﺘﺎﺑﻌﺔ اﻟﻤﻮﻇﻔﻴﻦ ﻓﻲ اﻟﻤﺴﺘﺸﻔﻰ ﺧﻠﻴﻞ ﻛﺎﻋﻴﻦ، ﻓﻲ ﺑﻴﺎن، ﺻﻴﺪا ووزﻳﺮ اﻟﺼﺤﺔ ﺟﻤﻴﻞ ﺟﺒﻖ واﻟﻤﺴﺆوﻟﻴﻦ اﻟﻤﻌﻨﻴﻴﻦ وﻓﺎﻋﻠﻴﺎت ﻣﺪﻳﻨﺔ ﺻﻴﺪا ﻟﻠﺘﺪﺧﻞ واﻧﺼﺎف اﻟﻤﻮﻇﻔﻴﻦ ودﻓﻊ ﻟﻢ ﻳﺴﺘﻄﻴﻌﻮا اﻟﺪوام ، ﻣﺸﻴﺮاً إﻟﻰ أنّ اﻟﻤﻮﻇﻔﻴﻦ رواﺗﺒﻬﻢ اﻟﻤﺴﺘﺤﻘﺔ وﻋﺪم اﻟﺴﻤﺎح ﺑﺈﻗﻔﺎل اﻟﻤﺴﺘﺸﻔﻰ "\n    cleaned text : " اﻋﺗﺻﺎم ﻟﻣوظﻔﻲ ﻣﺳﺗﺷﻔﻰ ﺻﯾدا اﻟﺣﻛوﻣﻲ اﺣﺗﺟﺎﺟﺎً ﻋﻠﻰ ﻋدم دﻓﻊ ﻣﺳﺗﺣﻘﺎﺗﮭم أﻣﺎم ﺑﺎﺣﺔ ﻣﺪﺧﻞ اﻟﻄﻮارىءاﺣﺘﺠﺎﺟﺎً ﻋﻠﻰ ﻋﺪم دﻓﻊ رواﺗﺒﻬﻢ ﻣﻨﺬ ﺧﻤﺴﺔ أﺷﻬﺮ وﺗﻀﺎﻣﻨﺎً ﻣﻊ زﻣﻴﻠﻬﻢ ﻣﺤﻤﺪ ﻣﺼﻄﻔﻰ اﻟﻤﻀﺮب ﻋﻦ اﻟﻄﻌﺎم ﻣﻨﺬ ﻳﻮﻣﻴﻦ رﺋﻴﺲ اﻟﺤﻜﻮﻣﺔ ﺳﻌﺪ اﻟﺤﺮﻳﺮي اﺑﻦ وﻧﺎﺷﺪ رﺋﻴﺲ ﻟﺠﻨﺔ ﻣﺘﺎﺑﻌﺔ اﻟﻤﻮﻇﻔﻴﻦ ﻓﻲ اﻟﻤﺴﺘﺸﻔﻰ ﺧﻠﻴﻞ ﻛﺎﻋﻴﻦ، ﻓﻲ ﺑﻴﺎن، ﺻﻴﺪا ووزﻳﺮ اﻟﺼﺤﺔ ﺟﻤﻴﻞ ﺟﺒﻖ واﻟﻤﺴﺆوﻟﻴﻦ اﻟﻤﻌﻨﻴﻴﻦ وﻓﺎﻋﻠﻴﺎت

In [9]:
def gpt_clean(text):
    prompt = f"""
    Your task is to Proofread and correct the Arabic text inside the double quotation marks,the text was extracted from a PDF page that has only one article, so the text might include irrelevant article titles or repeated information so make sure to exclude those and return only one relevant text without any extra information 
    and you have to consider correcting the misspiled words,
    as well as adding space between words if needed, grammar correction, and delete duplicated sentences
    you need to correct the folowing text and return only the corrected version: "{text}"
    """
    response = get_completion(prompt)
    return response

In [10]:
text = 'اﻟﻧﮭﺎر ﺑﯾن اﻟﮭﺟرة واﻟﻠﺟوء اﻹﻧﺳﺎﻧﻲ ﻟﺑﻧﺎﻧﯾون وﻓﻠﺳطﯾﻧﯾون ﻋﻠﻰ أﺑواب اﻟﺳﻔﺎرات ﻟﺗﺣﻘﯾﻖ اﻟﺣﻠم اﻗرأﻓﻲﺑرﯾﻣﯾوماﻟﻧﮭﺎرﺑﯾناﻟﮭﺟرةواﻟﻠﺟوءاﻹﻧﺳﺎﻧﻲﻟﺑﻧﺎﻧﯾونوﻓﻠﺳطﯾﻧﯾونﻋﻠﻰأﺑواب ﻟﺒﻨﺎﻧﻴﻮن وﻓﻠﺴﻄﻴﻨﻴﻮن ﻋﻠﻰ ﺑﻴﻦ اﻟﻬﺠﺮة واﻟﻠﺠﻮء اﻹﻧﺴﺎﻧﻲ أﺑﻮاب اﻟﺴﻔﺎرات ﻟﺘﺤﻘﻴﻖ اﻟﺤﻠﻢ اﻟﻨﻬﺎر اﻟﻤﺼﺪر أﺳﺮار ﺷﺒﺎرو ﻣﺸﻬﺪ ﻏﻴﺮ ﻣﺄﻟﻮف ﻳﻌﻜﺲ ﺣﺎﻟﺔ اﻟﻴﺄس واﻹﺣﺒﺎط اﻟﺘﻲ ﻳﻌﻴﺸﻬﺎ اﻟﺸﺒﺎب اﻟﻠﺒﻨﺎﻧﻲ واﻟﻔﻠﺴﻄﻴﻨﻲ ﻋﻠﻰ ﺣﺪﱟ ﺳﻮاء، ﻇﻬﺮ أﻣﺎم اﻟﺴﻔﺎرة اﻟﻜﻨﺪﻳﺔ ﻓﻲ ﺟﻞ اﻟﺪﻳﺐ، ﻣﻦ ﺧﻼل اﻋﺘﺼﺎﻣﺎت ﻣﻄﺎﻟﺒﺔ ﺑﻔﺘﺢ ﺑﺎب اﻟﻬﺠﺮة واﻟﻠﺠﻮء اﻹﻧﺴﺎﻧﻲ، ﻻ ﺗﻘﺒﻞ ﺗﻘﺪﻳﻢ اﻟﻄﻠﺒﺎت ﻣﺒﺎﺷﺮة، ﺷﺨﺼﻴﺎً أو ﻋﻦ ﻃﺮﻳﻖ اﻟﺒﺮﻳﺪ اﻹﻟﻜﺘﺮوﻧﻲ، اﻷﻣﺮ اﻟﺬي دﻓﻌﻬﺎ إﻟﻰ إﻋﻼن أﻧّﻬﺎ اﻟﺴﻔﺎرة اﻟﻜﻨﺪﻳﺔ ﻟﻴﺴﺖ إﻻ ﺑﺎﺑﺎً ﻹﻋﺎدة اﻟﺘﻮﻃﻴﻦ أو ﻷي ﻧﻮع آﺧﺮ ﻣﻦ أﻧﻮاع اﻟﻬﺠﺮة اﻗﺘﺼﺎدﻳﺔ أو ﻋﺎﺋﻠﻴﺔ أو إﻧﺴﺎﻧﻴﺔ ﻣﻦ أﺑﻮاب ﻋﺪة ﺳﻴﻄﺮﻗﻬﺎ ﻣﻦ اﺗﺨﺬوا اﻟﻘﺮار ﺑﻤﻐﺎدرة ﻟﺒﻨﺎن، واﻷﻳﺎم اﻟﻤﻘﺒﻠﺔ ﺳﺘﺸﻬﺪ ﻣﺰﻳﺪاً ﻣﻦ اﻟﺤﺸﻮد واﻻﻋﺘﺼﺎﻣﺎت أﻣﺎم ﺳﻔﺎرات ﻋﺪة، إﻟﻰ أن ﻳﻔﺘﺢ ﺑﺎب إﺣﺪاﻫﺎ وﻳﺘﻤﻜﻦ اﻟﺸﺒﺎب ﻣﻦ اﻻﻧﺘﻘﺎل إﻟﻰ دوﻟﺔ أﺟﻨﺒﻴﺔ ﺗﺤﻘﻖ ﻣﻦ رﺣﻢ اﻷﻟﻤﻔﻜﺮة اﻻﻋﺘﺼﺎم أﻣﺎم اﻟﺴﻔﺎرة اﻟﻜﻨﺪﻳﺔ واﻟﻤﻄﺎﻟﺒﺔ ﺑﺎﻟﻬﺠﺮة ﺗﻌﻮد ﻛﻤﺎ ﻗﺎل ﺣﻠﻤﻬﻢ ﺑﺎﻟﻌﻴﺶ اﻟﻜﺮﻳﻢ ﻟﺒﻨﺎﻧﻴﻮن وﻓﻠﺴﻄﻴﻨﻴﻮن ﻋﻠﻰ أﺑﻮاب اﻟﺴﻔﺎرات ﻟﺘﺤﻘﻴﻖ اﻟﺤﻠﻢ ﺑﻴﻦ اﻟﻬﺠﺮة واﻟﻠﺠﻮء اﻹﻧﺴﺎﻧﻲ'

In [11]:
# gpt_clean(text)

In [14]:
cell = worksheet.find(text, in_column=160)
worksheet.update_cell(cell.row, 165, 'cleaned_text') 

{'spreadsheetId': '1p64cFLW4g6ENO8i6462NkORnYFPThZxBK8mscPgl_Vg',
 'updatedRange': 'Main!FI134',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [12]:
count = 0
cleaned_values = []
for text in texts:
    try:
        cell = worksheet.find(text, in_column=160)
        if cell.row > 131: 
            cleaned_text = gpt_clean(text)
            cleaned_values.append(cleaned_text)
            worksheet.update_cell(cell.row, 165, cleaned_text)
            count += 1
            print(f"{round((count * 100)/len(texts),2)} %")
    except:
        cell = worksheet.find(text, in_column=160)
        worksheet.update_cell(cell.row, 165, 'error')
        count += 1
        print(f"{round((count * 100)/len(texts),2)} %")

0.05 %
0.09 %
0.14 %
0.19 %
0.24 %
0.28 %


In [5]:
# def clean_text(text):
#     new_text = []
#     for x in text.split():
#         x = ''.join(i for i in x if i not in string.digits and i not in string.ascii_letters and i not in string.punctuation)
#         x = x.replace('�','')
#         new_text.append(x)
#     return ' '.join(x for x in new_text if x)

In [20]:
def clean_text(text):
    new_text = []
    for x in text.split():
        #x = ''.join(i for i in x if i not in string.digits and i not in string.ascii_letters and i not in string.punctuation)
        x = x.replace('\ue140','').replace('�','').replace('�','').replace('','')
        new_text.append(x)
    return ' '.join(x for x in new_text if x)

In [21]:
# Clean the text in the column
count = 0
#cleaned_values = []
for text in texts:
    try:
        cleaned_text = clean_text(text)
        cells_text = worksheet.findall(text, )
        for cell in cells_text:
            worksheet.update_cell(cell.row, 164, cleaned_text)
        count += 1
        print(f"{round((count * 100)/len(texts),2)} %")
        
    except:
        continue

0.23 %
0.47 %
0.7 %
0.93 %
1.16 %
1.4 %
1.63 %
1.86 %
2.09 %
2.33 %
2.56 %
2.79 %
3.02 %
3.26 %
3.49 %
3.72 %
3.95 %
4.19 %
4.42 %
4.65 %
4.88 %
5.12 %
5.35 %
5.58 %
5.81 %
6.05 %
6.28 %
6.51 %
6.74 %
6.98 %
7.21 %
7.44 %
7.67 %
7.91 %
8.14 %
8.37 %
8.6 %
8.84 %
9.07 %
9.3 %
9.53 %
9.77 %
10.0 %
10.23 %
10.47 %
10.7 %
10.93 %
11.16 %
11.4 %
11.63 %
11.86 %
12.09 %
12.33 %
12.56 %
12.79 %
13.02 %
13.26 %
13.49 %
13.72 %
13.95 %
14.19 %
14.42 %
14.65 %
14.88 %
15.12 %
15.35 %
15.58 %
15.81 %
16.05 %
16.28 %
16.51 %
16.74 %
16.98 %
17.21 %
17.44 %
17.67 %
17.91 %
18.14 %
18.37 %
18.6 %
18.84 %
19.07 %
19.3 %
19.53 %
19.77 %
20.0 %
20.23 %
20.47 %
20.7 %
20.93 %
21.16 %
21.4 %
21.63 %
21.86 %
22.09 %
22.33 %
22.56 %
22.79 %
23.02 %
23.26 %
23.49 %
23.72 %
23.95 %
24.19 %
24.42 %
24.65 %
24.88 %
25.12 %
25.35 %
25.58 %
25.81 %
26.05 %
26.28 %
26.51 %
26.74 %
26.98 %
27.21 %
27.44 %
27.67 %
27.91 %
28.14 %
28.37 %
28.6 %
28.84 %
29.07 %
29.3 %
29.53 %
29.77 %
30.0 %
30.23 %
30.47 %
30.7 %
30